# SSY340 Project - Trajectory Estimation

### Fill in group number and member names:

In [1]:
NAME1 = "Bingcheng Chen" 
NAME2 = "Arvin Rokni"
GROUP = "Project groups 64"

## 1. Loading the data

In [ ]:
import model
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
from torch import nn
import numpy as np
from tqdm import tqdm 
from torch.utils.data import DataLoader

## 2. Pre-processing

## 3. Data loaders

## 4. Training

#### 4.1 create a model

In [ ]:
transformer_model = model.Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

#### 4.2 The training loop

In [ ]:
def train_epoch(model, optimizer, loss_fn, train_loader, val_loader, device, print_every):
    # Train:
    model.train()
    
    train_loss_batches = []

    num_batches = len(train_loader)

    for batch_index, data in enumerate(train_loader):

        # prepare the input
        encoder_input = 0
        decoder_input = 0
        decoder_source_mask = 0
        decoder_target_mask = 0

        optimizer.zero_grad()
        predictions = transformer_model.forward(encoder_input, decoder_input) # add mask ?

        loss = F.pairwise_distance()
        

        loss.backward()
        optimizer.step()

        train_loss_batches.append(loss.item())

        # If you want to print your progress more often than every epoch you can
        # set `print_every` to the number of batches you want between every status update.
        # Note that the print out will trigger a full validation on the full val. set => slows down training
        if print_every is not None and batch_index % print_every == 0:
            val_loss = validate(model, loss_fn, val_loader, device)
            model.train()
            print(f"\tBatch {batch_index}/{num_batches}: "
                  f"\tTrain loss: {sum(train_loss_batches[-print_every:])/print_every:.3f}, "
                  f"\tVal. loss: {val_loss:.3f}, "

    return model, train_loss_batches

def validate(model, loss_fn, val_loader, device):
                    
    with torch.no_grad():

        batch_val_loss=0
        gt = []
        pr = []
        model.eval()

        for id_b, data in enumerate(val_loader):
            # storing groung truth 
            gt.append(data['trg'][:, :, 0:2])

            # input to encoder input
            val_input = (data['src'][:,1:,2:4].to(device)-mean.to(device))/std.to(device)

            # input to decoder
            start_of_seq = torch.Tensor([0, 0, 1]).unsqueeze(0).unsqueeze(1).repeat(val_input.shape[0], 1, 1).to(device)
            dec_inp = start_of_seq
            # decoder masks
            dec_source_mask = torch.ones((val_input.shape[0], 1, val_input.shape[1])).to(device)
            dec_target_mask = utils.subsequent_mask(dec_inp.shape[1]).repeat(dec_inp.shape[0], 1, 1).to(device)

            # prediction till horizon lenght
            for i in range(horizon):
                # getting model prediction
                model_output = tf_model.forward(val_input, dec_inp, dec_source_mask, dec_target_mask)
                # appending the predicition to decoder input for next cycle
                dec_inp = torch.cat((dec_inp, model_output[:, -1:, :]), 1)

            # calculating loss using pairwise distance of all predictions
            val_loss = F.pairwise_distance(dec_inp[:,1:,0:2].contiguous().view(-1, 2),
                                    ((data['trg'][:, :, 2:4].to(device)-mean.to(device))/std.to(device)).\
                                        contiguous().view(-1, 2).to(device)).mean() + \
                                        torch.mean(torch.abs(dec_inp[:,1:,2]))
            batch_val_loss += val_loss.item()

            # calculating the position for each time step of prediction based on velocity
            preds_tr_b = (dec_inp[:, 1:, 0:2]*std.to(device) + mean.to(device)).cpu().numpy().cumsum(1) + \
                data['src'][:,-1:,0:2].cpu().numpy()

            pr.append(preds_tr_b)
        validation_loss.append(batch_val_loss/len(val_loader))

        # calculating mad and fad evaluation metrics
        gt = np.concatenate(gt, 0)
        pr = np.concatenate(pr, 0)
        mad, fad, _ = dataloader.distance_metrics(gt, pr)
        val_mad.append(mad)
        val_fad.append(fad)

        print("Epoch {}/{}....Validation mad = {:.4f}, Validation fad = {:.4f}".format(epoch+1, epochs, mad, fad))

def training_loop(model, optimizer, loss_fn, train_loader, val_loader, num_epochs, print_every):
    print("Starting training")
    device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
    model.to(device)
    train_losses, val_losses = [], []

    for epoch in range(1, num_epochs+1):
        model, train_loss, train_acc = train_epoch(model,
                                                   optimizer,
                                                   loss_fn,
                                                   train_loader,
                                                   val_loader,
                                                   device,
                                                   print_every)
        val_loss, val_acc = validate(model, loss_fn, val_loader, device)
        print(f"Epoch {epoch}/{num_epochs}: "
              f"Train loss: {sum(train_loss)/len(train_loss):.3f}, "
              f"Train acc.: {sum(train_acc)/len(train_acc):.3f}, "
              f"Val. loss: {val_loss:.3f}, "
              f"Val. acc.: {val_acc:.3f}")
        train_losses.extend(train_loss)
        train_accs.extend(train_acc)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
    return model, train_losses, train_accs, val_losses, val_accs

#### 4.3 Train the model

In [ ]:
loss_fn = F.pairwise_distance()

optimizer = torch.optim.SGD(tf_model.parameters(), lr=1e-4, momentum=0.9, weight_decay=1e-3, nesterov=True)
num_epochs = 10

first_model, first_train_losses, first_train_accs, first_val_losses, first_val_accs = training_loop(model, optimizer, loss_fn, train_dataloader, val_dataloader, num_epochs=num_epochs, print_every=10)


#### 4.4 save the model

## 5. Visualisation

## 6. Evaluation on test set